In [1]:
from primeqa.components.retriever.dense import ColBERTRetriever
from primeqa.components.reader.prompt import PromptGPTReader
from primeqa.pipelines.qa_pipeline import QAPipeline
import json

index_root = "/dccstor/mabornea1/kilt-wikipedia-test/colbert_ir/kilt_wikipedia_eli5_dev_exp/indexes/"
index_name = "kilt_wikipedia_eli5_dev_indname"
collection = "/dccstor/mabornea1/kilt-wikipedia-test/passages/kilt_knowledgesource_eli5_dev.tsv"


retriever = ColBERTRetriever(index_root = index_root, 
                                     index_name = index_name, 
                                     collection = collection, 
                                     max_num_documents = 3)
retriever.load()

reader = PromptGPTReader(api_key="")
reader.load()

pipeline = QAPipeline(retriever, reader)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
{"time":"2023-02-03 12:55:30,072", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss."}
{"time":"2023-02-03 12:55:30,155", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss."}
[Feb 03, 12:55:32] #> base_config.py from_path /dccstor/mabornea1/kilt-wikipedia-test/colbert_ir/kilt_wikipedia_eli5_dev_exp/indexes//kilt_wikipedia_eli5_dev_indname/metadata.json
[Feb 03, 12:55:32] #> base_config.py from_path args loaded! 
[Feb 03, 12:55:32] #> base_config.py from_path args replaced ! 
[Feb 03, 12:55:40] #>>>>> at ColBERT name (model type) : /dccstor/colbert-ir/franzm/experiments/oct2_7_12_1.5e-06/none/2022-10/09/15.21.39/checkpoints/colbert.dnn.batch_91287.model
[Feb 03, 12:55:40] #>>>>> at BaseColBERT name (model type) : /dccstor/colbert-ir/franzm/experiments/oct2_7_12_1.5e-06/none/2022-10/09/15.21.39/checkpoints/colbert.dnn.batch_91287.model
[Feb 03, 12:55:54] factory model type: xlm-robe

/dccstor/mabornea1/miniconda3/envs/prompt_reader/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


[Feb 03, 12:56:27] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Feb 03, 12:56:28] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


14635it [00:00, 198680.84it/s]


In [2]:
questions = ["What causes the trail behind jets at high altitude?"]
answers = pipeline.run(questions)
print(json.dumps(answers, indent=4))

[Feb 03, 12:56:40] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[Feb 03, 12:56:40] #> Input: $ What causes the trail behind jets at high altitude?, 		 True, 		 None
[Feb 03, 12:56:40] #> Output IDs: torch.Size([32]), tensor([     0,   9748,   4865, 113660,     70, 141037,  50155,     55,    933,
            99,  11192,    144,  35810,     32,      2,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[Feb 03, 12:56:40] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[Feb 03, 12:56:40] #>>>> colbert query ==
[Feb 03, 12:56:40] #>>>>> input_ids: torch.Size([32]), tensor([     0,   9748,   4865, 113660,     70, 141037,  50155,     55,    933,
            99,  11192,    144,  35810,     32,      2,      1,      1,      1,
             1,      1,      1,      

/dccstor/mabornea1/miniconda3/envs/prompt_reader/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Feb 03, 12:56:40] #>>>> before linear query ==
[Feb 03, 12:56:40] #>>>>> Q: torch.Size([32, 1024]), tensor([[-0.9069, -0.0403,  1.6935,  ..., -2.0556, -0.3505,  0.3143],
        [-0.6235, -0.2982,  0.1217,  ...,  0.0441, -1.5926, -0.2264],
        [-0.4985, -0.5053, -0.0043,  ...,  0.1960, -1.3059, -0.4401],
        ...,
        [-1.5241, -0.1893,  0.4258,  ...,  0.2577, -1.6957, -0.0245],
        [-1.5241, -0.1893,  0.4258,  ...,  0.2577, -1.6957, -0.0245],
        [-1.5241, -0.1893,  0.4258,  ...,  0.2577, -1.6957, -0.0245]])
[Feb 03, 12:56:40] #>>>>> self.linear query : Parameter containing:
tensor([[-0.0301, -0.0307, -0.0115,  ..., -0.0231, -0.0023, -0.0216],
        [ 0.0053,  0.0023, -0.0308,  ...,  0.0108,  0.0011,  0.0201],
        [-0.0220,  0.0370,  0.0339,  ..., -0.0023, -0.0172,  0.0244],
        ...,
        [ 0.0222,  0.0115, -0.0246,  ...,  0.0389, -0.0034, -0.0165],
        [-0.0146,  0.0392,  0.0131,  ..., -0.0055,  0.0219, -0.0368],
        [ 0.0071,  0.0256, -0.0346

100%|██████████| 1/1 [00:00<00:00, 40.63it/s]


We do not currently have a prompt
[
    {
        "example_id": 0,
        "text": "This is a placeholder and we do not call the API"
    }
]
